In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

### userID-itemID matrix 생성(using Collab)

In [ ]:
user_df = pd.read_csv("랜덤유저데이터(협업필터링알고리즘에서사용).csv",encoding='utf-8',usecols=['userID','레시피북마크기록'])

In [ ]:
columns = ["userID","itemID","eventStrength"]
df_cf_item = pd.DataFrame([],columns=columns)

count = 0
for index,row in user_df.iterrows():
    userID = row['userID']
    items_ID = row['레시피북마크기록']
    for itemID in eval(items_ID):
      df_cf_item = df_cf_item.append(pd.Series([userID,itemID,1],index=columns),ignore_index=True)
      count += 1
      if count % 10000 == 0:
        print(count)

In [ ]:
from google.colab import files
df_cf_item.to_csv('user_recipe_matrix.csv') # collab에서 생성된 csv파일 로컬로 다운로드
files.download('user_recipe_matrix.csv')

### df matrix 로컬로 불러오기 

In [2]:
user_recipe_matrix = pd.read_csv('user_recipe_matrix.csv',index_col='Unnamed: 0')
user_recipe_matrix.rename(columns={'eventStrength':'Quantity'},inplace=True)
user_recipe_matrix.head() # 유저별로 특정 itemID를 가진 제품에 좋아요를 눌렀는지 여부
item_lookup = pd.DataFrame(user_recipe_matrix['itemID'].drop_duplicates())
user_recipe_matrix = user_recipe_matrix.groupby(['userID','itemID']).sum().reset_index()
user_recipe_matrix.head()

,userID,itemID,Quantity
0,0,9,1
1,0,12,1
2,0,77,1
3,0,78,1
4,0,80,1


### 모델 학습에 쓰일 희소행렬 형태의 matrix (데이터프레임 -> 희소행렬 변환)

In [3]:
# 행을 유저ID, 열을 itemID로 두고, 좋아요수량(Quantity)를 행렬값으로 두는 희소행렬을 만든다
users = list(np.sort(user_recipe_matrix['userID'].unique()))
items = list(user_recipe_matrix['itemID'].unique())
quantity = list(user_recipe_matrix['Quantity'])

rows = user_recipe_matrix['userID'].astype('category').cat.codes
cols = user_recipe_matrix['itemID'].astype('category').cat.codes

purchase_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(users),len(items)))
purchase_sparse.shape # 100명의 고객 & 14678개의 아이템으로 이루어짐. 유저/아이템간 상호작용은 100*14678.

(1000, 1317)

In [4]:
# 상호작용 행렬의 희소성 확인
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity # (sparsity < 99.5)

96.26826119969628

In [7]:
# Train/Validation 세트 만들기
# user-item matrix에서, 훈련 데이터는 percentage 비율(20%)만큼 0으로 바뀔 것임. 

def make_train (matrix, percentage = .2):
    test_set = matrix.copy() # 원본 유저-아이템 행렬의 복사본
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs))) 
    samples = random.sample(nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples] # 훈련 데이터에서 0으로 바뀐 유저의 index
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros() # 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    
    return training_set, test_set, list(set(user_inds))

# 훈련, 테스트 데이터 생성
product_train, product_test, product_users_altered = make_train(purchase_sparse, 0.2)

In [8]:
# 모델 생성 

alpha = 15 # ALS모델의 objective function이 MSE error & (모델의 복잡도/과접합 조정)L2항을 사용. 손실함수에 a알파를 곱해서 가중치를 조정

user_vecs, item_vecs = implicit.alternating_least_squares( # user벡터, item벡터 
    (product_train*alpha).astype('double')
    , factors=100
    , regularization = 0.1 
    , iterations = 100)

predictions = [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)] # factorized시킨 user벡터와 item벡터를 내적하면 예측 행렬을 구할 수 있음. 

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████████████████████████████████████████████████████████████████████████| 100.0/100 [00:08<00:00, 10.45it/s]


In [9]:
from sklearn import metrics

# AUC를 계산하는 함수 
def auc_score (test, predictions):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr,tpr)

# 가려진 정보가 있는 유저마다 AUC 평균을 구하는 함수
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    input
    1. training_set: 훈련 데이터 (일정 비율로 아이템 구매량이 0으로 가려진 데이터)
    2. prediction: implicit MF에서 나온 유저/아이템 별로 나온 예측 평점 행렬
    3. altered_users: make_train 함수에서 아이템 구매량이 0으로 가려진 유저
    4. test_set: make_train함수에서 만든 테스트 데이터
    ----------------------------------------
    반환
    추천 시스템 유저의 평균 auc
    '''
    # 리스트 초기화
    store_auc = []    

    item_vecs = predictions[1] # 아이템 latent 벡터
    
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1) # 유저의 훈련데이터
        zero_inds = np.where(training_row == 0) # 가려진 아이템 Index
        
        # 가려진 아이템에 대한 예측
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # 가려진 아이템에 대한 실제값
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # AUC 계산 
        store_auc.append(auc_score(actual, pred))
    
    return float('%.3f'%np.mean(store_auc))

In [10]:
calc_mean_auc(product_train, product_users_altered, predictions, product_test)
# ALS 기반 추천시스템은 평균 0.485의 AUC를 갖음. 
# 예상 원인) 
# 1) 한 유저가 관심도일 보일 것 같은 아이템 수에 비해, 현재 가지고 있는 items수가 30000개.
# 2) np,randrange 메소드를 이용해 유저가 좋아요를 누른 제품 리스트를 임의로 만듦. np.randrange는 biased한 output를 내보낸다고 함. 

0.503

In [11]:
# 유저의 추천 아이템 반환 함수
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    INPUT
    1. customer_id - Input the customer's id number that you want to get recommendations for
    2. mf_train: 훈련 데이터
    3. user_vecs: 행렬 분해에 쓰인 유저 벡터
    4. item_vecs: 행렬 분해에 쓰인 아이템 벡터
    5. customer_list: 평점 행렬의 행에 해당하는 고객 ID
    6. item_list: 평점 행렬의 열에 해당하는 아이템 ID
    7. item_lookup: 아이템 ID와 설명을 담은 테이블
    8. num_items: 추천할 아이템 개수
    -----------------------------------------------------
    반환    
    구매한 적이 없는 아이템 중 예측 평점이 높은 최고 n개의 추천 아이템
    '''
    cust_ind = np.where(customer_list == customer_id)[0][0]
    pref_vec = mf_train[cust_ind,:].toarray()                   # 훈련 데이터의 실제 좋아요여부
    pref_vec = pref_vec.reshape(-1)                         
    pref_vec[pref_vec > 1] = 0                                  # 좋아요누른 것들을 모두 0으로 
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T)         # 추천 시스템에 기반한 예측 평점
    
    # Min-Max Scaling
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled  # 구매하지 않은 아이템에 대해서만 예측 평점이 남도록
    
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # num_items만큼 내림차순으로 평점 정렬한 index
    
    rec_list = []
    
    for index in product_idx:
        code = item_list[index] # 아이템 id
        # id 담기
        rec_list.append([code]) 
    
    codes = [item[0] for item in rec_list]
    final_frame = pd.DataFrame({'itemID': codes})
    
    return final_frame[['itemID']]

In [14]:
user_ID = 999

In [15]:
# 모델이 예측한 99번 유저의 좋아요를 누르지 않은 아이템 중 추천할 아이템을 뽑는 함수
rec_items(user_ID, product_train, user_vecs, item_vecs, np.array(users), np.array(items), item_lookup, num_items = 10)

,itemID
0,718
1,254
2,249
3,540
4,401
5,504
6,479
7,736
8,1136
9,838
